# Exploration

In [115]:
import nltk
import pandas as pd
import re
import string
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import sklearn
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  WordPunctTokenizer
from nltk.tokenize import  BlanklineTokenizer

In [ ]:
nltk.download("stopwords")
stop_words = nltk.corpus.stopwords.words("english")

In [2]:
df = pd.read_csv("data/labeled_data.csv", sep = ',').drop("Unnamed: 0", axis=1)
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
df.describe()

,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,3.243473,0.280515,2.413711,0.549247,1.110277
std,0.883060,0.631851,1.399459,1.113299,0.462089
min,3.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,2.000000,0.000000,1.000000
50%,3.000000,0.000000,3.000000,0.000000,1.000000
75%,3.000000,0.000000,3.000000,0.000000,1.000000
max,9.000000,7.000000,9.000000,9.000000,2.000000


In [4]:
df.loc[20000]

count                                                                 3
hate_speech                                                           0
offensive_language                                                    3
neither                                                               0
class                                                                 1
tweet                 RT @shananigans__: 50 shades of shut your bitc...
Name: 20000, dtype: object

In [19]:
df["count"].value_counts()

3    22807
6     1571
4      211
9      167
7       27
Name: count, dtype: int64

In [20]:
df["hate_speech"].value_counts()

0    19790
1     3419
2     1251
3      287
4       21
5        7
6        5
7        3
Name: hate_speech, dtype: int64

In [26]:
df[df["hate_speech"] == 7]["tweet"][6171]

'@infidelpamelaLC I\'m going to blame the black man, since they always blame "whitey" I\'m an equal opportunity hater.'

In [27]:
df.loc[6171]

count                                                                 9
hate_speech                                                           7
offensive_language                                                    1
neither                                                               1
class                                                                 0
tweet                 @infidelpamelaLC I'm going to blame the black ...
Name: 6171, dtype: object

## FRASE TEST

In [90]:
text = df["tweet"][146]
text

'"@ItsYahBoiRay: @anelylove if I don\'t get my dick sucked at yo party by a bad bitch I\'m fr gone set it off &#128514;&#128514;&#128514;&#128514;" *by @anelylove *'

## Tokenization

In [20]:
tt = TweetTokenizer()
wpt = WordPunctTokenizer()
blt = BlanklineTokenizer()

In [21]:
print(tt.tokenize(text.lower())) # Tweet
print(wpt.tokenize(text.lower())) # Wordpunct
print(blt.tokenize(text.lower())) # Blankline

['"', "don't", 'make', 'me', 'make', 'you', 'fall', 'in', 'live', 'with', 'a', 'nigga', 'like', 'meee', '...', '"', 'the', 'birds', '1', '&', '2', 'are', 'my', 'favorite', 'songs', 'by', 'weeknd']
['"', 'don', "'", 't', 'make', 'me', 'make', 'you', 'fall', 'in', 'live', 'with', 'a', 'nigga', 'like', 'meee', '..."', 'the', 'birds', '1', '&', 'amp', ';', '2', 'are', 'my', 'favorite', 'songs', 'by', 'weeknd']
['"don\'t make me make you fall in live with a nigga like meee..." the birds 1&amp;2 are my favorite songs by weeknd']


## Removing stopwords

In [25]:
nltk.download("stopwords")
stop_words = nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
tokens_text = tt.tokenize(text)
remove_sw = []
for token in tokens_text:
    if token.lower() not in stop_words:
         remove_sw.append(token)
print(remove_sw)

['"', 'make', 'make', 'fall', 'live', 'nigga', 'like', 'meee', '...', '"', 'birds', '1', '&', '2', 'favorite', 'songs', 'Weeknd']


## Remove numbers

Questa parte va inserita prima della rimozione delle stopwords

In [30]:
remove_num = re.sub(r'\d+', '', text)
print(remove_num)

"Don't make me make you fall in live with a nigga like meee..." The birds &amp; are my favorite songs by Weeknd


## Remove punctuations

Ovviamente pure sta parte va fatta prima

In [83]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [31]:
remove_punc = text.translate(str.maketrans('', '', string.punctuation))
print(remove_punc)

Dont make me make you fall in live with a nigga like meee The birds 1amp2 are my favorite songs by Weeknd


## Remove Extra spaces

Direi parecchio utile

In [32]:
remove_sp = re.sub(r'\s\s', '', text)
print(remove_sp)

"Don't make me make you fall in live with a nigga like meee..." The birds 1&amp;2 are my favorite songs by Weeknd


## WordCloud

In [ ]:
def plot_wordcloud(cnt, file_name="figure1.png"):
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white")
    wordcloud.generate_from_frequencies(dict(cnt))
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    # plt.show()
    plt.savefig(file_name)

## Stemming

In [ ]:


tokenized_text = WordPunctTokenizer().tokenize(text)

porter = PorterStemmer()
stemmed = [porter.stem(word) for word in tokenized_text]

## Stanford

In [37]:
import stanfordnlp
# stanfordnlp.download('en')   # This downloads the English models. Run only once!
base = "C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\"
config = {
    'processors': 'tokenize,pos,lemma,depparse', # Comma-separated list of processors to use
    'lang': 'en', # Language code for the language to build the Pipeline in
    'model_path': base + 'en_ewt_tokenizer.pt', 
    'pos_model_path': base + 'en_ewt_tagger.pt', 
    'pos_pretrain_path': base + 'en_ewt.pretrain.pt', 
    'lemma_model_path': base + 'en_ewt_lemmatizer.pt', 
    'depparse_model_path': base + 'en_ewt_parser.pt', 
    'depparse_pretrain_path': base + 'en_ewt.pretrain.pt', 
    'tokenize_pretokenized': False # Use pretokenized text as input and disable tokenization
}

nlp = stanfordnlp.Pipeline(**config)

doc = nlp("this is a simple text")
all_information_tab_separated = doc.conll_file.conll_as_string()

parsed_text = "\n".join([f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\tdepencency: {word.dependency_relation}' for sent in doc.sentences for word in sent.words])

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'pretokenized': False, 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt_parser.pt', 'pretrain_pa

C:\ProgramData\Anaconda3\lib\site-packages\stanfordnlp\models\depparse\model.py:157: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:650.)
  unlabeled_scores.masked_fill_(diag, -float('inf'))


In [41]:
print(dir(doc))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_conll_file', '_sentences', '_text', 'conll_file', 'load_annotations', 'sentences', 'text', 'write_conll_to_file']


In [59]:
print(dir(doc.sentences[0].words[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_dependency_relation', '_feats', '_governor', '_index', '_lemma', '_parent_token', '_text', '_upos', '_xpos', 'dependency_relation', 'feats', 'governor', 'index', 'lemma', 'parent_token', 'pos', 'text', 'upos', 'xpos']


In [55]:
doc.sentences[0].words[0].text

'this'

In [57]:
doc.sentences[0].words[1].lemma

'be'

In [67]:
doc.sentences[0].words[2].upos

'DET'

## Functions

In [99]:
def preprocessing(text):
    text = text.lower() # Lowering case
    remove_sp = re.sub(r'\s\s', '', text) # Removing exceeding spaces
    remove_retweet = re.sub(r"@\w+", "",remove_sp) # Removing retweet
    remove_punc = remove_retweet.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
    final_text = re.sub(r'\d+', '', remove_punc) # Remove number 
    final_text = re.sub(r'\s\s', '', final_text) # Removing exceeding spaces
    return final_text

In [100]:
print(text)
preprocessing(text)

"@ItsYahBoiRay: @anelylove if I don't get my dick sucked at yo party by a bad bitch I'm fr gone set it off &#128514;&#128514;&#128514;&#128514;" *by @anelylove *


'if i dont get my dick sucked at yo party by a bad bitch im fr gone set it offby'

In [106]:
def tokenization(text_clean, tok = "tweet"):
    if tok == "tweet": # TweetTokenizer
        tt = TweetTokenizer()
        tokenized_text = tt.tokenize(text_clean)
    elif tok == "wordpunct": # WordPunctTokenizer
        wpt = WordPunctTokenizer()
        tokenized_text = wpt.tokenize(text_clean)
    return tokenized_text

In [107]:
def remove_stopwords(tokenized_text):
    remove_sw = []
    for token in tokenized_text:
        if token.lower() not in stop_words:
             remove_sw.append(token)
    return remove_sw

In [108]:
def stemmer(tokenized_text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in tokenized_text]

In [ ]:
base = "C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\"
config = {
    'processors': 'tokenize,pos,lemma,depparse', # Comma-separated list of processors to use
    'lang': 'en', # Language code for the language to build the Pipeline in
    'model_path': base + 'en_ewt_tokenizer.pt', 
    'pos_model_path': base + 'en_ewt_tagger.pt', 
    'pos_pretrain_path': base + 'en_ewt.pretrain.pt', 
    'lemma_model_path': base + 'en_ewt_lemmatizer.pt', 
    'depparse_model_path': base + 'en_ewt_parser.pt', 
    'depparse_pretrain_path': base + 'en_ewt.pretrain.pt', 
    'tokenize_pretokenized': False # Use pretokenized text as input and disable tokenization
}

nlp = stanfordnlp.Pipeline(**config)

doc = nlp("this is a simple text")
all_information_tab_separated = doc.conll_file.conll_as_string()

parsed_text = "\n".join([f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\tdepencency: {word.dependency_relation}' for sent in doc.sentences for word in sent.words])
def stanford_parser(text):
    

In [128]:
preprocessing(text)

'if i dont get my dick sucked at yo party by a bad bitch im fr gone set it offby'

In [138]:
nlp(preprocessing(text))

RuntimeError: index_select(): Expected dtype int64 for index

In [ ]:
# lemmatization ?
# stemming ?
# normalization ?
# n-grams ?

# representation (word2vec - gensim)
# text clustering/classification
# evaluation con confronti
# conclusioni